<a href="https://colab.research.google.com/github/Engbasemhamada/Hotel-Booking-EDA/blob/main/task2_of_hotel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 2. Load Dataset
data = pd.read_csv("first inten project.csv")  # غيّر الاسم حسب الملف بتاعك
data

In [ ]:
# 3. Data Preprocessing
print(" Data Info:")
print(data.info())

# Missing values
print("\n Missing Values:")
print(data.isnull().sum())

# Strip spaces in column names
data.columns = data.columns.str.strip()

# Fill missing numeric values with median
data = data.fillna(data.median(numeric_only=True))


In [ ]:
# 4. Handle Outliers (IQR Method)
def remove_outliers_iqr(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    df = df[(df[col] >= lower) & (df[col] <= upper)]
    return df

num_cols = data.select_dtypes(include=np.number).columns
for col in num_cols:
    data = remove_outliers_iqr(data, col)
data

In [ ]:
X = data.drop("booking status", axis=1)
y = data["booking status"]

# لو الهدف نصي → نحوله لأرقام
if y.dtype == "object":
    le = LabelEncoder()
    y = le.fit_transform(y)

# اختياري: اختيار أفضل الميزات (هنا 5 مبدئيًا)
if len(num_cols) > 2:  # نتأكد فيه أعمدة كفاية
    selector = SelectKBest(chi2, k=min(5, len(num_cols)))
    X_selected = selector.fit_transform(X[num_cols], y)
else:
    X_selected = X[num_cols]

# PCA لتقليل الأبعاد (اختياري)
if X_selected.shape[1] > 2:
    pca = PCA(n_components=2)
    X_reduced = pca.fit_transform(X_selected)
else:
    X_reduced = X_selected


In [ ]:
# 6. Transform Categorical Data
# ==============================
cat_cols = X.select_dtypes(include="object").columns
le = LabelEncoder()
for col in cat_cols:
    X[col] = le.fit_transform(X[col])


In [ ]:
# 7. Scaling
# ==============================
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# 8. Train-Test Split
# ==============================
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [ ]:
# 📌 8. Modeling
# ==============================
# Logistic Regression
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)
y_pred_lr = log_reg.predict(X_test)

# Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)


In [ ]:
# 📌 9. Evaluation
# ==============================
print("\n📈 Logistic Regression")
print("Accuracy:", accuracy_score(y_test, y_pred_lr))
print(confusion_matrix(y_test, y_pred_lr))
print(classification_report(y_test, y_pred_lr))

print("\n🌳 Random Forest")
print("Accuracy:", accuracy_score(y_test, y_pred_rf))
print(confusion_matrix(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))